In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import re, string, time, random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
from kaggle_datasets import KaggleDatasets
import os, csv, collections
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv(r'/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv', parse_dates=['date'])
test_df = pd.read_csv(r'/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
df['item_cnt_day'] = df['item_cnt_day'].clip(0,20)
df = df[df['shop_id'].isin(test_df['shop_id'].unique())]
df = df[df['item_id'].isin(test_df['item_id'].unique())]
df = df.pivot_table(index=['shop_id', 'item_id'], values=['item_cnt_day'], columns=['date_block_num']
                     , fill_value=0, aggfunc='sum')
df = df.reset_index()

In [ ]:
df

In [ ]:
print(len(df.columns[12:-12]))
print(len(df.columns[-12:]))

In [ ]:
series_value=[]
list0=list(df.columns[12:-12])
list0.insert(0, (     'shop_id', ''))
list0.insert(1, (     'item_id', ''))
list1=list(df.columns[-12:])
list1.insert(0, (     'shop_id', ''))
list1.insert(1, (     'item_id', ''))
for idx in df.index:
    for i in range(2):
        if i==0:
            series_value.append(np.expand_dims(df.loc[idx, list0], 0))
        if i==1:
            series_value.append(np.expand_dims(df.loc[idx, list1], 0))
series_value = np.array(series_value)
series_value = series_value.reshape(-1,14)

In [ ]:
columns=['shop_id','item_id'] 
[columns.append(i) for i in range(11)]
columns.append('label')
df = pd.DataFrame(series_value, columns=columns)

In [ ]:
df_value = df.drop(columns=['shop_id', 'shop_id'])
df_value

In [ ]:
#label = df['label']
#df.pop('label')
#np_df = np.expand_dims(df, axis=-1)

In [ ]:
#dataset = tf.data.Dataset.from_tensor_slices((np_df, label.values))
#dataset = dataset.window(window_size+1, shift=1, drop_remainder=True)
#dataset = dataset.flat_map(lambda x:x.batch(window_size+1))

In [ ]:
def window_dataset(df, window_size, batch_size, shuffle_buffer):
    df = tf.expand_dims(df, axis=-1)
    dataset = tf.data.Dataset.from_tensor_slices([df])
    dataset = dataset.window(window_size+1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda x:x.batch(window_size+1))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.map(lambda x:(x[:-1,:], x[-1,:]))
    return dataset.batch(batch_size)

In [ ]:
#window_size=29
#batch_size=32
#shuffle_buffer=5
#dataset = window_dataset(df.values, window_size, batch_size, shuffle_buffer)

In [ ]:
df_value.values.shape

In [ ]:
df[1::2]

In [ ]:
train_x = np.expand_dims(np.array(df_value)[:,:-1], axis=-1)
train_y = df_value['label'].values
#test_y = np.expand_dims(np.array(df_value)[1::2,1:], axis=-1)
print(train_x.shape)
print(train_y.shape)

In [ ]:
df_value

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(10, input_shape=(11,1), return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(10),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(optimizer='adam', loss='mae', metrics=['mse'])

In [ ]:
history = model.fit(train_x, train_y, validation_split=0.1, epochs=5)

In [ ]:
plt.plot(np.arange(len(history.history['loss'])), history.history['loss'])
plt.plot(np.arange(len(history.history['val_loss'])), history.history['val_loss'])
plt.legend(['loss', 'val_loss'])

In [ ]:
plt.plot(np.arange(len(history.history['mse'])), history.history['mse'])
plt.plot(np.arange(len(history.history['val_mse'])), history.history['val_mse'])
plt.legend(['mse', 'val_mse'])

In [ ]:
test_x = pd.merge(test_df, df[1::2], on=['shop_id', 'item_id'], how='left')
test_x.fillna(0, inplace=True)
test_x = test_x.drop(columns=['ID', 'shop_id', 'item_id', 0]).values
test_x = np.expand_dims(test_x, axis=-1)

In [ ]:
value = model.predict(test_x)
test_df['item_cnt_month'] = value
test_df.pop('shop_id')
test_df.pop('item_id')
test_df.to_csv(r'submission.csv', index=False)